In [86]:
import os
import re
import csv
import time 
import random 
import sqlite3
import keyboard 
import funcionesAux as fc
from datetime import datetime
from bs4 import BeautifulSoup
from seleniumbase import Driver
from selenium.webdriver.common.by import By

def dia_csv(datos, nombre_archivo = "dia.csv"):
    """
    Guarda los datos en un archivo CSV.

    Parámetros:
    datos -- Lista de diccionarios con los datos a guardar.
    nombre_archivo -- Nombre del archivo CSV a crear.
    """

    # Nombres de las columnas
    columnas = datos[0].keys()

    # Comprueba si el archivo existe
    existe_archivo = os.path.isfile(nombre_archivo)

    # Abre el archivo en modo 'a+' (append) si existe, o en modo 'w' (write) si no existe
    with open(nombre_archivo, 'a+' if existe_archivo else 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=columnas)

        # Escribe el encabezado solo si el archivo no existía
        if not existe_archivo:
            writer.writeheader()

        # Escribe los datos en el archivo CSV
        writer.writerows(datos)

In [2]:
driver = Driver(
    browser="chrome",
    uc=True,
    headless2=False,
    incognito=False,
    agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
    do_not_track=True,
    undetectable=True
)

driver.maximize_window()

fecha = datetime.now().date()
print(f"Iniciando escaneo a fecha: {datetime.now()}")

# Diccionario para guardar los anuncios
dia_anuncios = {}
datos = []

# El escaneo se inicia por categorías. Itera sobre cada una, empezando por la primera
url = "https://www.dia.es/charcuteria-y-quesos/jamon-cocido-lacon-fiambres-y-mortadela/c/L2001"
driver.get(url)
time.sleep(random.uniform(3,5))
# Acepta cookies
fc.click_element(driver, By.ID, 'onetrust-accept-btn-handler')

lista_categorias = []
categorias = fc.wait_for_elements(driver, By.CSS_SELECTOR, '[data-test-id="categories-list-element"]', multiple=True)
print(f"Categorías: {len(categorias)}") # 23 categorías

for categoria in categorias: 
    print(categoria)
    categoria.click()
    time.sleep(1)
    # Selecciona todos los elementos con el atributo 'data-test-id="sub-category-item"'
    sub_categories = fc.wait_for_elements(categoria, By.CSS_SELECTOR, 'div[data-test-id="sub-category-item"]', multiple=True)
    
    html = categoria.get_attribute("innerHTML")
    soup = BeautifulSoup(html, 'html.parser')

    # Encuentra todos los elementos <a> con el atributo 'data-test-id="sub-category-item-link"'
    links = soup.find_all('a', attrs={'data-test-id': 'sub-category-item-link'})

    # Para cada enlace, obtén su href
    hrefs = [link.get('href') for link in links]
    print(f"\nSubcategorías en {hrefs[0].split("/")[1]}: {len(sub_categories)}")
    # Imprime todos los hrefs

    for href in hrefs:
        print("https://www.dia.es" + href)
        lista_categorias.append("https://www.dia.es" + href)
        driver.get("https://www.dia.es" + href)
        time.sleep(random.uniform(3,6))
        
        # Por cada enlace tenemos una lista de productos
        time.sleep(random.uniform(2,3))
        el = fc.wait_for_elements(driver, By.CSS_SELECTOR, '.product-container', multiple=False)
        html_articulos = el.get_attribute("innerHTML")
        soup_articulos = BeautifulSoup(html_articulos, 'html.parser')
        print(soup_articulos)

        # Para cada articulo
        list_items = soup_articulos.find_all('div', attrs={'data-test-id': 'product-card'})
        for list in list_items:
            print(list)
            a_element = list.find('a')
            enlace = a_element.get('href')
            img_element = a_element.find('img')  # Encuentra la etiqueta <img> dentro de la etiqueta <a>
            imagen = img_element.get('src')  # Obtiene el atributo src de la etiqueta <img>
            p_titulo = list.find('p', class_='search-product-card__product-name')
            titulo = p_titulo.text
            p_element = list.find('p', attrs={'data-test-id': 'search-product-card-unit-price'})
            price = p_element.text
            price = price.replace(".","").replace(",",".").replace("€", "").strip()
            pk_element = list.find('p', attrs={'data-test-id': 'search-product-card-kilo-price'})
            precio_kilo = pk_element.text
            precio_kilo = precio_kilo.replace(".","").replace(",",".").replace("€", "").replace("/KILO").replace("(","").replace(")","").strip()
            print(f"Titulo: {titulo}\nEnlace: {enlace}\nimagen: {imagen}\nPrecio: {price}\nPrecio por kilo: {precio_kilo}")

        # p_element = soup.find('p', class_='search-product-card__product-name')
        # text = p_element.text

    
print(lista_categorias)

Iniciando escaneo a fecha: 2024-05-03 15:54:22.730519
